In [1]:
import base64
import getpass
import json

import requests

In [2]:
username = 'alice'
password = username.upper()

In [3]:
CAS = 'http://localhost:7000'
RM = 'http://localhost:8002'
AR = 'http://localhost:8003'

In [5]:
response = requests.post(CAS + '/auth/tokens', json={'username': username, 'password': password})
token = response.json()['token']

In [6]:
response = requests.get(AR + '/sites/')
assert response.status_code == 200
sites = response.json()

In [7]:
response = requests.post(AR + '/sites/', 
    headers={'Dibbs-Authorization': token},
    json={
        'name': 'roger',
        'type': 'openstack',
        'api_url': 'http://roger-openstack.ncsa.illinois.edu:5000/v2.0',
    },
)
assert response.status_code == 201
site = response.json()

In [8]:
apps = requests.get(AR + '/appliances/').json()

In [9]:
#requests.delete(AR + '/appliances/{}/'.format(apps[0]['id']),
#                headers={'Dibbs-Authorization': token},
#)

In [10]:
response = requests.post(AR + '/appliances/', 
    headers={'Dibbs-Authorization': token},
    json={
        'name': 'hadoop',
        'description': 'Named after an Elephant from the San Diego zoo',
    },
)
assert response.status_code == 201
app = response.json()

In [12]:
with open('appliances/hadoop-roger.yml', 'r') as f:
    template = f.read()

response = requests.post(AR + '/implementations/', 
    headers={'Dibbs-Authorization': token},
    json={
        'appliance': app['id'],
        'site': site['id'],
        'script': template
    },
)
assert response.status_code == 201
imp = response.json()

In [13]:
import subprocess

def obfuscate(data):
    return base64.b64encode(json.dumps(data).encode('utf-8')).decode('utf-8')

roger_pass = subprocess.check_output('ssh roger "cat ~/.openstack"', shell=True, universal_newlines=True).strip()
creds = {
    'username': 'npt',
    'password': roger_pass,
    'project_name': 'DIBBs',
}
cred_string = obfuscate(creds)

In [14]:
response = requests.post(RM + '/credentials/',
    headers={'Dibbs-Authorization': token},
    json={
        'name': 'me-keys',
        'site': site['id'],
        'credentials': cred_string,
    }
)
assert response.status_code == 201
creds = response.json()

In [15]:
response = requests.post(RM + '/clusters/',
    headers={'Dibbs-Authorization': token},
    json={
        'credential': creds['id'],
        'implementation': imp['id'],
    }
)
assert response.status_code == 201
cluster = response.json()

In [16]:
cluster

{'address': '',
 'credential': 'b5080ecb-7156-42ef-bf63-a851258edab4',
 'id': 'ab69db79-4a5a-4e96-aff0-b1abb3410f51',
 'implementation': '3b3770b4-28fd-48d3-a338-4f7225dc11b0',
 'remote_id': 'faebced3-e85c-4f05-91a5-08ac179b350f',
 'remote_status': 'CREATE_IN_PROGRESS',
 'root_owner': 1,
 'status': 'BUILDING'}

In [25]:
response = requests.delete(RM + '/clusters/{}/'.format(cluster['id']),
    headers={'Dibbs-Authorization': token},
)
response.status_code

204

In [32]:
requests.get(RM + '/clusters/').json()

[{'address': '',
  'credential': 'b5080ecb-7156-42ef-bf63-a851258edab4',
  'id': '221e4305-d98c-4f43-9953-31087230c606',
  'implementation': '3b3770b4-28fd-48d3-a338-4f7225dc11b0',
  'remote_id': '257cbb79-89a9-4275-bb4f-6b508b0918d6',
  'remote_status': 'CREATE_IN_PROGRESS',
  'root_owner': 1,
  'status': 'BUILDING'}]

In [27]:
response = requests.post(RM + '/resources/', 
    headers={'Dibbs-Authorization': token},
    json={
        'hints': {
            'implementation': imp['id'],
            'credentials': creds['id'],
        },
    }
)
assert response.status_code == 201
resource = response.json()

In [33]:
resource

{'cluster': '221e4305-d98c-4f43-9953-31087230c606',
 'hints': '{"implementation": "3b3770b4-28fd-48d3-a338-4f7225dc11b0", "credentials": "b5080ecb-7156-42ef-bf63-a851258edab4"}',
 'id': '4e19eb20-7209-4c35-be7b-a481972a7ea6',
 'password': '',
 'user': 1,
 'username': ''}

In [34]:
response = requests.get(RM + '/resources/{}/'.format(resource['id']))
response.status_code

200

In [35]:
response.json()

{'cluster': '221e4305-d98c-4f43-9953-31087230c606',
 'hints': '{"implementation": "3b3770b4-28fd-48d3-a338-4f7225dc11b0", "credentials": "b5080ecb-7156-42ef-bf63-a851258edab4"}',
 'id': '4e19eb20-7209-4c35-be7b-a481972a7ea6',
 'password': '1hgOTOXaE34gCz4TUFoH',
 'user': 1,
 'username': 'user_YFrbKiTNyWSovMPX3cT7Rl1aD'}